<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/uso_fm_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instala dependências

In [ ]:
!pip install "torch>=2.0" "transformers>=4.44" "datasets>=3.0" scikit-learn accelerate

## Inferência com o BERT (sem treino)

In [ ]:
# inferencia_bert.py
from transformers import pipeline

# 1) Preenchimento de máscara (masked language modeling) em EN
mlm = pipeline("fill-mask", model="bert-base-uncased")
print(mlm("Paris is the [MASK] of France.")[:3])  # top-3 sugestõe

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


[{'score': 0.9969332218170166, 'token': 3007, 'token_str': 'capital', 'sequence': 'paris is the capital of france.'}, {'score': 0.0005914872162975371, 'token': 2540, 'token_str': 'heart', 'sequence': 'paris is the heart of france.'}, {'score': 0.0004378740268293768, 'token': 2415, 'token_str': 'center', 'sequence': 'paris is the center of france.'}]


In [ ]:
# 2) Classificação de sentimento em PT-BR usando um BERT treinado em português
# (pode trocar por outro checkpoint de PT-BR disponível no Hub)
clf = pipeline("text-classification", model="neuralmind/bert-base-portuguese-cased", tokenizer="neuralmind/bert-base-portuguese-cased")
print(clf("O serviço foi excelente, recomendo muito!"))
print(clf("O produto chegou quebrado, experiência péssima."))

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.5278118848800659}]
[{'label': 'LABEL_0', 'score': 0.5345523953437805}]


# Exemplo completo: fine-tuning de BERT para classificação de sentimentos (PT-BR)

In [ ]:
# treino_bert_pt_classificacao.py
import numpy as np
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
    TextClassificationPipeline,
)

def build_tiny_pt_dataset():
    # 0 = NEGATIVE, 1 = POSITIVE
    textos = [
        "O produto é excelente e chegou antes do prazo.",
        "Atendimento impecável, voltarei a comprar.",
        "Horrível, veio com defeito e ninguém responde.",
        "A embalagem veio danificada e o suporte foi ruim.",
        "Funciona muito bem, qualidade acima do esperado.",
        "Péssima experiência, não recomendo a ninguém.",
        "Entrega rápida e o material é muito bom.",
        "Demorou demais, muito frustrante.",
        "Superou minhas expectativas, cinco estrelas!",
        "Não atendeu ao que foi prometido."
    ]
    rotulos = [1,1,0,0,1,0,1,0,1,0]
    ds = Dataset.from_dict({"text": textos, "label": rotulos})
    split = ds.train_test_split(test_size=0.3, seed=42)  # 70/30
    return split

def tokenize_function(batch, tokenizer):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

def compute_metrics(eval_pred):
    preds = eval_pred.predictions[0] if isinstance(eval_pred.predictions, tuple) else eval_pred.predictions
    preds = np.argmax(preds, axis=1)
    acc = accuracy_score(eval_pred.label_ids, preds)
    f1 = f1_score(eval_pred.label_ids, preds)
    return {"accuracy": acc, "f1": f1}

def main():
    set_seed(42)
    model_name = "neuralmind/bert-base-portuguese-cased"

    # 1) Dados
    ds = build_tiny_pt_dataset()

    # 2) Tokenizer e dataset tokenizado
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenized = ds.map(lambda b: tokenize_function(b, tokenizer), batched=True)
    tokenized = tokenized.remove_columns(["text"])
    tokenized = tokenized.rename_column("label", "labels")
    tokenized.set_format("torch")

    id2label = {0: "NEGATIVE", 1: "POSITIVE"}
    label2id = {"NEGATIVE": 0, "POSITIVE": 1}

    # 3) Modelo
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        id2label=id2label,
        label2id=label2id,
    )

    # 4) Configuração de treino
    args = TrainingArguments(
        output_dir="./bert-pt-classifier",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,        # aumente para dados reais
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        logging_steps=5,
        report_to="none",          # evita logs no wandb
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # 5) Treino e avaliação
    trainer.train()
    print(trainer.evaluate())

    # 6) Salvar e inferir
    trainer.save_model("./bert-pt-classifier")
    pipe = TextClassificationPipeline(model=trainer.model, tokenizer=tokenizer, return_all_scores=False)
    exemplos = [
        "O atendimento foi rápido e eficiente.",
        "Muito ruim, não funcionou como esperado.",
        "Cumpriu o prometido, estou satisfeito."
    ]
    for t in exemplos:
        print(t, "->", pipe(t)[0])

In [ ]:
main()

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from transformers import pipeline

qa = pipeline("question-answering", model="deepset/bert-base-cased-squad2", tokenizer="deepset/bert-base-cased-squad2")
qa({"question": "Quem escreveu Dom Casmurro?",
    "context": "Dom Casmurro é um romance do escritor brasileiro Machado de Assis."})


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.3178841769695282, 'start': 18, 'end': 25, 'answer': 'romance'}

In [ ]:
from transformers import pipeline

ner = pipeline("token-classification", model="dslim/bert-base-NER", aggregation_strategy="simple")
ner("Hugging Face is based in New York City.")

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity_group': 'ORG',
  'score': np.float32(0.76597357),
  'word': 'Hugging Face',
  'start': 0,
  'end': 12},
 {'entity_group': 'LOC',
  'score': np.float32(0.9994878),
  'word': 'New York City',
  'start': 25,
  'end': 38}]

# Exemplo de uso do GTP

In [ ]:
!pip install "torch>=2.0" "transformers>=4.44" "datasets>=3.0" "accelerate>=0.34" peft

In [ ]:
# gpt_inferencia.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def main_gpt2():
    model_name = "gpt2"  # troque por outro checkpoint se quiser
    device = "cuda" if torch.cuda.is_available() else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # GPT-2 não tem pad_token por padrão: use o EOS como PAD
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    model.eval()

    prompt = (
        "Explique, em poucas linhas, o que é aprendizado de máquina e cite um exemplo prático."
        "\nResposta:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,       # amostragem (criatividade)
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

In [ ]:
main_gpt2()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Explique, em poucas linhas, o que é aprendizado de máquina e cite um exemplo prático.
Resposta: ¿Que español? Quando diferente en la suación alcune a estudiantivo un entendido con sus segúncias para lo cuales y el bienvenidos por cuatro recuerdos también del mejor frentivamentar así perdida (sicieron) inépidas las comunidad siempre no sería llegada exigués desde nombres-sanitaria! Esperanza vieja hacia


# GPT peft

In [ ]:
# gpt_finetune_lora.py
import math
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    set_seed,
)
from peft import LoraConfig, get_peft_model, TaskType
import transformers

def build_tiny_instruct_ds():
    # Mini dataset de exemplos (INSTRUÇÃO -> RESPOSTA)
    pairs = [
        ("Explique o que é overfitting em aprendizado de máquina.",
         "Overfitting é quando o modelo aprende demais os detalhes do conjunto de treino, incluindo ruídos, perdendo capacidade de generalização. Usa-se validação, regularização e mais dados para mitigar."),
        ("Liste três vantagens de redes neurais profundas.",
         "1) Capturam padrões complexos; 2) Funcionam bem com dados não estruturados; 3) Podem ser refinadas com pretreino e fine-tuning."),
        ("O que é gradiente descendente?",
         "É um método iterativo de otimização que ajusta parâmetros na direção oposta ao gradiente do erro para minimizar a função de perda."),
        ("Dê um exemplo de uso de aprendizado por reforço.",
         "Treinar um agente a jogar xadrez aprendendo políticas por meio de recompensas ao vencer partidas e punições por movimentos ruins."),
        ("Explique batch size em treinamento de redes neurais.",
         "É o número de exemplos processados antes de atualizar os pesos. Batches maiores tendem a estimar melhor o gradiente, mas consomem mais memória."),
    ]
    # Formato simples estilo “instrução-resposta”
    texts = [f"### Instrução:\n{q}\n\n### Resposta:\n{a}\n" for q, a in pairs]
    ds = Dataset.from_dict({"text": texts})
    return ds.train_test_split(test_size=0.4, seed=42)  # treino/val pequeno

def tokenize_fn(batch, tokenizer, max_len=512):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=max_len,
    )

def main_peft():
    set_seed(42)
    print("Transformers versão:", transformers.__version__)
    model_name = "gpt2"  # pode trocar por outro GPT compatível
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 1) Dataset (didático)
    ds = build_tiny_instruct_ds()

    # 2) Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenized = ds.map(lambda b: tokenize_fn(b, tokenizer), batched=True)
    tokenized = tokenized.remove_columns(["text"])
    tokenized.set_format("torch")

    # 3) Modelo base + LoRA
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    base_model.resize_token_embeddings(len(tokenizer))  # garante pad/EOS
    # Configuração LoRA (leve e suficiente p/ demo)
    peft_cfg = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["c_attn", "c_fc", "c_proj"],  # camadas comuns do GPT-2
    )
    model = get_peft_model(base_model, peft_cfg).to(device)
    model.print_trainable_parameters()

    # 4) Data collator para LM causal (rótulos = input_ids)
    collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # 5) TrainingArguments (com fallback p/ versões antigas)
    base_args = dict(
        output_dir="./gpt2-pt-lora",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,  # economiza VRAM
        learning_rate=2e-4,
        num_train_epochs=3,             # aumente em dados reais
        weight_decay=0.01,
        logging_steps=10,
        report_to=[],                   # evita integração externa
        fp16=(device == "cuda"),        # meia precisão na GPU
        save_total_limit=2,
    )
    try:
        args = TrainingArguments(
            **base_args,
            evaluation_strategy="steps",
            eval_steps=20,
            save_strategy="steps",
            save_steps=20,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
        )
    except TypeError:
        print("Aviso: TrainingArguments compatível (sem evaluation/save_strategy).")
        args = TrainingArguments(**base_args)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        data_collator=collator,
    )

    # 6) Treinar + avaliar
    trainer.train()
    metrics = trainer.evaluate()
    print("Métricas de validação:", metrics)
    if "eval_loss" in metrics:
        try:
            ppl = math.exp(metrics["eval_loss"])
            print(f"Perplexidade aprox.: {ppl:.2f}")
        except OverflowError:
            pass

    # 7) Geração com o modelo ajustado (sem precisar mesclar pesos)
    model.eval()
    prompt = (
        "### Instrução:\nExplique o que é regularização L2.\n\n### Resposta:\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    print(tokenizer.decode(out[0], skip_special_tokens=True))

    # 8) Salvar adapter LoRA
    trainer.model.save_pretrained("./gpt2-pt-lora")  # salva os adapters
    tokenizer.save_pretrained("./gpt2-pt-lora")


In [ ]:
main()

Explique, em poucas linhas, o que é aprendizado de máquina e cite um exemplo prático.
Resposta: es un trabajo en sus autes con el segunda (I'm here to help you). No sécurité diferente et la vida y no estos añar entre los serraçones ("It's important that we do not go back and kill the dead"). Dios nos llegadas nada para águido "cómo", cosa hacer dos perdías comención algunamientoes son escribir-de-más sobre non mi obrera del fuese porque


# GPT Lora

In [ ]:
# usar_adapter.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base = "gpt2"
adapter_dir = "./gpt2-pt-lora"
device = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(adapter_dir)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base_model = AutoModelForCausalLM.from_pretrained(base).to(device)
model = PeftModel.from_pretrained(base_model, adapter_dir).to(device)
model.eval()

prompt = "### Instrução:\nDefina aprendizado supervisionado.\n\n### Resposta:\n"
x = tok(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    y = model.generate(**x, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9,
                       pad_token_id=tok.pad_token_id, eos_token_id=tok.eos_token_id)
print(tok.decode(y[0], skip_special_tokens=True))

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './gpt2-pt-lora'.

In [ ]:
!pip install "torch>=2.0" "transformers>=4.44" "datasets>=3.0" "accelerate>=0.34" peft

In [ ]:
# gpt_lora_end2end.py
# Uso:
#   pip install "torch>=2.0" "transformers>=4.44" "datasets>=3.0" "accelerate>=0.34" peft
#   python gpt_lora_end2end.py
#   # opções:
#   #   python gpt_lora_end2end.py --force-retrain
#   #   python gpt_lora_end2end.py --model gpt2 --save-dir gpt2_pt_lora

import os
import math
import argparse
from pathlib import Path

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    set_seed,
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

# ---------- utilidades de dispositivo ----------
def pick_device():
    if torch.cuda.is_available():
        return "cuda"
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

# ---------- dataset didático ----------
def build_tiny_instruct_ds():
    pairs = [
        ("Explique o que é overfitting em aprendizado de máquina.",
         "Overfitting é quando o modelo aprende detalhes e ruído do treino e perde generalização. Mitigue com validação, regularização e mais dados."),
        ("Liste três vantagens de redes neurais profundas.",
         "Capturam padrões complexos; funcionam bem com dados não estruturados; permitem pré-treino e fine-tuning."),
        ("O que é gradiente descendente?",
         "Método iterativo que ajusta parâmetros na direção oposta ao gradiente para minimizar a perda."),
        ("Dê um exemplo de uso de aprendizado por reforço.",
         "Treinar um agente a jogar xadrez com recompensas por vitórias e punições por más jogadas."),
        ("Explique batch size em treinamento.",
         "Quantidade de exemplos antes de atualizar os pesos; batches maiores estimam melhor o gradiente, mas consomem mais memória."),
    ]
    texts = [f"### Instrução:\n{q}\n\n### Resposta:\n{a}\n" for q, a in pairs]
    ds = Dataset.from_dict({"text": texts})
    return ds.train_test_split(test_size=0.4, seed=42)

def tokenize_fn(batch, tokenizer, max_len=512):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_len)

# ---------- checagens de diretório ----------
def adapter_files_exist(save_dir: Path) -> bool:
    """Checa se os principais arquivos de adapter/PEFT existem."""
    expected = [
        save_dir / "adapter_config.json",
        save_dir / "adapter_model.safetensors",
    ]
    return all(p.exists() for p in expected)

# ---------- treino + salvamento ----------
def train_and_save(model_name: str, save_dir: Path, device: str):
    from transformers import DataCollatorForLanguageModeling

    set_seed(42)

    # dados
    ds = build_tiny_instruct_ds()

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenized = ds.map(lambda b: tokenize_fn(b, tokenizer), batched=True)
    tokenized = tokenized.remove_columns(["text"])
    tokenized.set_format("torch")

    # modelo base + LoRA
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    base_model.resize_token_embeddings(len(tokenizer))  # garante pad/eos
    peft_cfg = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["c_attn", "c_fc", "c_proj"],  # GPT-2 típico
    )
    model = get_peft_model(base_model, peft_cfg).to(device)
    model.print_trainable_parameters()

    collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    base_args = dict(
        output_dir=str(save_dir),
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=10,
        report_to=[],
        fp16=(device == "cuda"),
        save_total_limit=2,
    )

    # fallback para versões antigas do Transformers
    try:
        args = TrainingArguments(
            **base_args,
            evaluation_strategy="steps",
            eval_steps=20,
            save_strategy="steps",
            save_steps=20,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
        )
    except TypeError:
        print("Aviso: TrainingArguments compatível (sem evaluation/save_strategy).")
        args = TrainingArguments(**base_args)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        data_collator=collator,
    )

    trainer.train()
    metrics = trainer.evaluate()
    print("Métricas de validação:", metrics)
    if "eval_loss" in metrics:
        try:
            ppl = math.exp(metrics["eval_loss"])
            print(f"Perplexidade aprox.: {ppl:.2f}")
        except OverflowError:
            pass

    save_dir.mkdir(parents=True, exist_ok=True)
    trainer.model.save_pretrained(str(save_dir))   # salva adapters LoRA
    tokenizer.save_pretrained(str(save_dir))       # salva tokenizer

# ---------- carregamento + geração ----------
def load_and_generate(model_name: str, save_dir: Path, device: str):
    # Preferimos o tokenizer salvo junto ao adapter
    tokenizer = AutoTokenizer.from_pretrained(str(save_dir)) \
                if (save_dir / "tokenizer_config.json").exists() \
                else AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    base_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    model = PeftModel.from_pretrained(base_model, str(save_dir)).to(device)
    model.eval()

    prompt = "### Instrução:\nExplique o que é regularização L2.\n\n### Resposta:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    print("\n--- Saída gerada ---")
    print(tokenizer.decode(out[0], skip_special_tokens=True))

# ---------- main ----------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, default="gpt2", help="Checkpoint base (decoder-only).")
    parser.add_argument("--save-dir", type=str, default="gpt2_pt_lora", help="Diretório para salvar/carregar adapters.")
    parser.add_argument("--force-retrain", action="store_true", help="Força retreino mesmo se já houver adapters.")
    args = parser.parse_args()

    device = pick_device()
    print(f"Dispositivo: {device}")
    save_dir = Path(args.save_dir).resolve()
    print(f"Pasta de adapters: {save_dir}")

    if args.force_retrain or not adapter_files_exist(save_dir):
        if not args.force_retrain:
            print("Adapters não encontrados. Iniciando treino...")
        else:
            print("Forçando retreino por --force-retrain.")
        train_and_save(args.model, save_dir, device)
    else:
        print("Adapters encontrados. Pulando treino.")

    # carrega e gera (sempre roda)
    load_and_generate(args.model, save_dir, device)


usage: colab_kernel_launcher.py [-h] [--model MODEL] [--save-dir SAVE_DIR]
                                [--force-retrain]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-e65d0936-846f-45be-a32c-28777cb7f1e7.json


SystemExit: 2